# Packages

In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.0 MB/s eta 0:00

# 1. Cree la base de donnees

In [4]:
import chromadb

def init_db():
    """Initialisation de la base de données ChromaDB"""
    return chromadb.PersistentClient(path="/content/chromadb")

db = init_db()

# 2. Cree les Collections

In [5]:
roles = ["Simple", "Dev", "Admin"]
collections = {role: db.get_or_create_collection(name=f"qa_{role}") for role in roles}

In [6]:
collections

{'Simple': Collection(name=qa_Simple),
 'Dev': Collection(name=qa_Dev),
 'Admin': Collection(name=qa_Admin)}

In [7]:
db.list_collections()

['qa_Dev', 'qa_Simple', 'qa_Admin']

In [8]:
def Num_Data():
  for coll in collections:
      print('Num ', coll,' : ', collections[coll].count())

In [9]:
Num_Data()

Num  Simple  :  16
Num  Dev  :  16
Num  Admin  :  16


# 3. Add/ Ask questions

In [10]:
def add_document(role: str, question: str, answer: str):
    """Ajoute une question-réponse dans la collection associée au rôle spécifié"""
    if role not in collections:
        print("Rôle invalide!")
        return

    collection = collections[role]
    collection.add(
        documents=[question],
        metadatas=[{"answer": answer}],
        ids=[f"{role}_{question}"]  # Identifiant unique basé sur le rôle
    )
    print(f"Ajouté avec succès dans la collection {role}")

In [11]:
seuil = 0.5
message_question_existante_autre_role = "Cette question associée à un autre rôle."
message_hors_contexte = "Cette question ne correspond à aucun des rôles définis."

In [12]:
def ask_autre_role(role: str, question: str):
    for rol in collections:
        if rol != role:
            collection = collections[rol]
            results = collection.query(query_texts=[question], n_results=1)

            if results and results.get("documents"):
                distance = results["distances"][0][0] if results["distances"] else None
                if distance is not None and distance <= seuil:
                    return True
    return False  # Ajoute un return par défaut pour éviter des erreurs

In [13]:
def ask_question(role: str, question: str):
    """Recherche la réponse en fonction du rôle donné"""
    if role not in collections:
        return "Rôle invalide!"

    collection = collections[role]
    results = collection.query(query_texts=[question], n_results=1)

    if results and results.get("documents"):
        distance = results["distances"][0][0] if results["distances"] else None

        if distance <= seuil:
          answer = results["metadatas"][0][0]["answer"] if results["metadatas"] else "Réponse non disponible"
          #return f"Réponse: {answer}, Pertinence (distance): {distance:.4f}"
          return answer

        else :
          temp = ask_autre_role(role, question)
          if temp:
            return message_question_existante_autre_role
          else:
            return message_hors_contexte
          #return "Seuil elvee !"

        #return f"Réponse: {answer}, Pertinence (distance): {distance:.4f}"

    return "Aucune réponse trouvée"

In [14]:
# Exemple d'utilisation
add_document("Simple", "Quel est le capital du Maroc?", "Rabat")
add_document("Dev", "Qu'est-ce que Python?", "Un langage de programmation interprété.")
add_document("Admin", "Comment redémarrer un serveur Linux?", "Utiliser la commande 'sudo reboot'.")

Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Dev


Ajouté avec succès dans la collection Admin


In [15]:
print(ask_question("Simple", "Quel est le capital du Maroc?"))  # Devrait retourner Rabat
print(ask_question("Dev", "Qu'est-ce que Python?"))  # Devrait retourner la réponse associée
print(ask_question("Admin", "Comment redémarrer un serveur Linux?"))  # Réponse Admin

Rabat
Un langage de programmation interprété.
Utiliser la commande 'sudo reboot'.


# 4. Insertion des donnees

## 4.1 Simple utilisateur

In [16]:
add_document("Simple", "Quelle est la capitale de la France ?", "Paris")
add_document("Simple", "Quel est le plus grand océan du monde ?", "Océan Pacifique")
add_document("Simple", "Combien de continents y a-t-il sur Terre ?", "7")
add_document("Simple", "Qui a écrit 'Les Misérables' ?", "Victor Hugo")
add_document("Simple", "Quelle est la monnaie officielle des États-Unis ?", "Dollar américain")
add_document("Simple", "Quel est l'animal national de l'Australie ?", "Kangourou")
add_document("Simple", "Quelle est la langue officielle du Brésil ?", "Portugais")
add_document("Simple", "Quelle est la hauteur du Mont Everest ?", "8 848 mètres")
add_document("Simple", "Dans quel pays se trouve le Taj Mahal ?", "Inde")
add_document("Simple", "Quelle est la couleur du drapeau du Japon ?", "Rouge et blanc")

Ajouté avec succès dans la collection Simple


Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple


Ajouté avec succès dans la collection Simple


Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple


Ajouté avec succès dans la collection Simple


Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple


Ajouté avec succès dans la collection Simple


In [17]:
collections['Simple'].count()

16

## 4.2 Dev utilisateur

In [18]:
add_document("Dev", "Qu'est-ce qu'une API ?", "Interface de programmation permettant aux applications de communiquer entre elles.")
add_document("Dev", "Quelle est la différence entre un tableau et une liste en Python ?", "Un tableau a une taille fixe, alors qu'une liste est dynamique.")
add_document("Dev", "Qu'est-ce qu'un framework ?", "Un ensemble d'outils et de bibliothèques permettant de développer des applications rapidement.")
add_document("Dev", "Quelle est la fonction de Git ?", "Git est un système de contrôle de version distribué.")
add_document("Dev", "Qu'est-ce qu'une base de données relationnelle ?", "Une base de données où les données sont organisées en tables.")
add_document("Dev", "À quoi sert un fichier `.env` ?", "Il est utilisé pour stocker des variables d'environnement sensibles.")
add_document("Dev", "Qu'est-ce que le versioning de code ?", "Le versioning permet de gérer les différentes versions d'un code source.")
add_document("Dev", "Quelle est la différence entre une méthode GET et POST en HTTP ?", "GET est utilisé pour récupérer des données, POST pour envoyer des données.")
add_document("Dev", "Que fait une instruction `if` en programmation ?", "Elle permet d'exécuter du code conditionnel.")
add_document("Dev", "Qu'est-ce que le principe DRY en développement ?", "DRY signifie 'Don't Repeat Yourself', éviter la duplication du code.")

Ajouté avec succès dans la collection Dev


Ajouté avec succès dans la collection Dev


Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev


Ajouté avec succès dans la collection Dev


Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev


Ajouté avec succès dans la collection Dev


Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev


In [19]:
collections['Dev'].count()

16

## 4.3 Admin utilisateur

In [20]:
add_document("Admin", "Comment installer Apache sur un serveur Linux ?", "Utiliser la commande 'sudo apt install apache2' sur une machine Ubuntu.")
add_document("Admin", "Que faire en cas de 'Disk Full' sur un serveur Linux ?", "Libérer de l'espace disque en supprimant des fichiers inutiles.")
add_document("Admin", "Comment vérifier les journaux système sur un serveur Linux ?", "Utiliser la commande 'journalctl' pour consulter les journaux.")
add_document("Admin", "Comment configurer un pare-feu sur Ubuntu ?", "Utiliser 'ufw' pour configurer un pare-feu.")
add_document("Admin", "Quelle commande permet de redémarrer un service sous Linux ?", "Utiliser la commande 'sudo systemctl restart <service>'.")
add_document("Admin", "Comment ajouter un utilisateur sur un serveur Linux ?", "Utiliser la commande 'sudo adduser <nom_utilisateur>'.")
add_document("Admin", "Comment configurer un serveur DNS sur Linux ?", "Modifier le fichier '/etc/resolv.conf' pour spécifier les serveurs DNS.")
add_document("Admin", "Que fait la commande `chmod` ?", "Elle permet de modifier les permissions d'un fichier ou d'un répertoire.")
add_document("Admin", "Comment configurer un serveur Nginx pour une application web ?", "Modifier les fichiers de configuration dans '/etc/nginx/sites-available' et activer le site.")
add_document("Admin", "Que faire pour sécuriser un serveur Linux contre les attaques DDoS ?", "Utiliser des outils comme fail2ban et configurer un pare-feu adapté.")

Ajouté avec succès dans la collection Admin


Ajouté avec succès dans la collection Admin


Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin


Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin


Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin


In [21]:
collections['Admin'].count()

16

## 4.4 Question communes

In [22]:
# Question 1
add_document("Simple", "Qu'est-ce que le cloud ?", "Un espace de stockage en ligne pour conserver des fichiers accessibles partout.")
add_document("Dev", "Qu'est-ce que le cloud ?", "Une infrastructure qui permet d'héberger et d'exécuter des applications à distance.")
add_document("Admin", "Qu'est-ce que le cloud ?", "Un ensemble de serveurs distants offrant des services de stockage, de calcul et de réseau.")

# Question 2
add_document("Simple", "Qu'est-ce qu'un serveur ?", "Un ordinateur qui stocke et partage des informations sur Internet.")
add_document("Dev", "Qu'est-ce qu'un serveur ?", "Une machine qui exécute des applications backend et gère les requêtes des clients.")
add_document("Admin", "Qu'est-ce qu'un serveur ?", "Un système informatique qui fournit des services à d'autres machines dans un réseau.")

# Question 3
add_document("Simple", "À quoi sert un mot de passe ?", "À protéger l'accès à un compte personnel ou à des informations sensibles.")
add_document("Dev", "À quoi sert un mot de passe ?", "À sécuriser l'accès aux bases de données, aux API et aux outils de développement.")
add_document("Admin", "À quoi sert un mot de passe ?", "À contrôler l'authentification des utilisateurs sur les systèmes et les serveurs.")

# Question 4
add_document("Simple", "Comment fonctionne Internet ?", "Internet fonctionne en connectant des ordinateurs entre eux grâce à un réseau mondial.")
add_document("Dev", "Comment fonctionne Internet ?", "Internet repose sur des protocoles comme HTTP et TCP/IP pour échanger des données entre serveurs et clients.")
add_document("Admin", "Comment fonctionne Internet ?", "Internet est basé sur une infrastructure réseau reliant des millions de serveurs et d'appareils via des routeurs et des commutateurs.")

# Question 5
add_document("Simple", "Qu'est-ce qu'un fichier ?", "Un document numérique contenant des informations, comme une image ou un texte.")
add_document("Dev", "Qu'est-ce qu'un fichier ?", "Une structure de données stockée sur un disque, pouvant contenir du code, des logs ou des bases de données.")
add_document("Admin", "Qu'est-ce qu'un fichier ?", "Un élément du système de fichiers qui stocke des données accessibles via des commandes comme 'ls' ou 'cat'.")

Ajouté avec succès dans la collection Simple


Ajouté avec succès dans la collection Dev


Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Simple


Ajouté avec succès dans la collection Dev


Ajouté avec succès dans la collection Admin


...

Ajouté avec succès dans la collection Dev


Ajouté avec succès dans la collection Admin


Ajouté avec succès dans la collection Simple


Ajouté avec succès dans la collection Dev


Ajouté avec succès dans la collection Admin


# 5. Tester les reponses

## 5.1 Simple Utilisateur

### 1.1 Question simple

In [ ]:
list_simple = [
    ("Quelle est la capitale de la France ?", "Paris"),
    ("Quel est le plus grand océan du monde ?", "Océan Pacifique"),
    ("Combien de continents y a-t-il sur Terre ?", "7"),
    ("Qui a écrit 'Les Misérables' ?", "Victor Hugo"),
    ("Quelle est la monnaie officielle des États-Unis ?", "Dollar américain"),
    ("Quel est l'animal national de l'Australie ?", "Kangourou"),
    ("Quelle est la langue officielle du Brésil ?", "Portugais"),
    ("Quelle est la hauteur du Mont Everest ?", "8 848 mètres"),
    ("Dans quel pays se trouve le Taj Mahal ?", "Inde"),
    ("Quelle est la couleur du drapeau du Japon ?", "Rouge et blanc")
]

In [ ]:
for question in list_simple:
  #print(question[0])
  reponse_reel = question[1]
  reponse_genere = ask_question("Simple", question[0])

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  Paris
Reponse genere par le system :  Réponse: Paris, Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Océan Pacifique
Reponse genere par le system :  Réponse: Océan Pacifique, Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  7
Reponse genere par le system :  Réponse: 7, Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Victor Hugo
Reponse genere par le system :  Réponse: Victor Hugo, Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Dollar américain
Reponse genere par le system :  Réponse: Dollar américain, Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Kangourou
Reponse genere par le system :  Réponse: Kangourou, Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Portugais
Reponse genere

### 1.2 Question simple avec une reformulation

In [ ]:
list_simple_re = [
    ("Quelle est la ville qui est la capitale de la France ?", "Paris"),
    ("Quel est l'océan ayant la plus grande superficie ?", "Océan Pacifique"),
    ("Combien de continents sont présents sur notre planète ?", "7"),
    ("Quel auteur a écrit le livre 'Les Misérables' ?", "Victor Hugo"),
    ("Quelle devise est utilisée aux États-Unis ?", "Dollar américain"),
    ("Quel est l'animal emblématique de l'Australie ?", "Kangourou"),
    ("Quel est le principal langage parlé au Brésil ?", "Portugais"),
    ("Quelle est l'altitude du Mont Everest ?", "8 848 mètres"),
    ("Où peut-on trouver le Taj Mahal ?", "Inde"),
    ("Quelles couleurs composent le drapeau japonais ?", "Rouge et blanc")
]

In [ ]:
for question in list_simple_re:
  #print(question[0])
  reponse_reel = question[1]
  reponse_genere = ask_question("Simple", question[0])

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  Paris
Reponse genere par le system :  Réponse: Paris, Pertinence (distance): 0.1360
--------------------------------------------------
Reel reponse :  Océan Pacifique
Reponse genere par le system :  Réponse: Océan Pacifique, Pertinence (distance): 0.4077
--------------------------------------------------
Reel reponse :  7
Reponse genere par le system :  Réponse: 7, Pertinence (distance): 0.4266
--------------------------------------------------
Reel reponse :  Victor Hugo
Reponse genere par le system :  Réponse: Victor Hugo, Pertinence (distance): 0.1649
--------------------------------------------------
Reel reponse :  Dollar américain
Reponse genere par le system :  Réponse: Dollar américain, Pertinence (distance): 0.4087
--------------------------------------------------
Reel reponse :  Kangourou
Reponse genere par le system :  Réponse: Kangourou, Pertinence (distance): 0.2429
--------------------------------------------------
Reel reponse :  Portugais
Reponse genere

### 1.3 Question dans d'autre role

In [ ]:
for question in list_dev:
  #print(question[0])
  reponse_reel = question[1]
  reponse_genere = ask_question("Simple", question[0])

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  Interface de programmation permettant aux applications de communiquer entre elles.
Reponse genere par le system :  Cette question associée à un autre rôle.
--------------------------------------------------
Reel reponse :  Un tableau a une taille fixe, alors qu'une liste est dynamique.
Reponse genere par le system :  Cette question associée à un autre rôle.
--------------------------------------------------
Reel reponse :  Un ensemble d'outils et de bibliothèques permettant de développer des applications rapidement.
Reponse genere par le system :  Cette question associée à un autre rôle.
--------------------------------------------------
Reel reponse :  Git est un système de contrôle de version distribué.
Reponse genere par le system :  Cette question associée à un autre rôle.
--------------------------------------------------
Reel reponse :  Une base de données où les données sont organisées en tables.
Reponse genere par le system :  Cette question associée à un autre 

### 1.4 Question Hors context

In [ ]:
list_questions_hors = [
    ("Quelle est la durée d'un jour sur Mars ?", "24 heures 39 minutes"),
    ("Qui a inventé l'ampoule électrique ?", "Thomas Edison"),
    ("Qu'est-ce que le Big Bang ?", "La théorie qui décrit l'origine de l'univers à partir d'un point très dense et chaud."),
    ("Quelle est la plus grande forêt du monde ?", "La forêt amazonienne"),
    ("Qu'est-ce qu'une éclipse solaire ?", "Un phénomène où la Lune passe entre la Terre et le Soleil."),
    ("Quel est l'animal le plus rapide du monde ?", "Le guépard"),
    ("Qu'est-ce que la gravité ?", "La force qui attire les objets vers le centre de la Terre."),
    ("En quelle année l'Homme a-t-il marché sur la Lune pour la première fois ?", "1969"),
    ("Quel est le pays le plus peuplé du monde ?", "La Chine"),
    ("Qu'est-ce que l'effet de serre ?", "Le phénomène par lequel certains gaz emprisonnent la chaleur dans l'atmosphère terrestre.")
]

In [ ]:
for question in list_questions_hors:
  #print(question[0])
  reponse_reel = question[1]
  reponse_genere = ask_question("Simple", question[0])

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  24 heures 39 minutes
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  Thomas Edison
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  La théorie qui décrit l'origine de l'univers à partir d'un point très dense et chaud.
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  La forêt amazonienne
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  Un phénomène où la Lune passe entre la Terre et le Soleil.
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  Le guépa

### 2.4 Questions communes

In [ ]:
for question in list_questions_communes:
  #print(question[0])
  reponse_reel = question
  reponse_genere = ask_question("Simple", question)

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  Qu'est-ce que le cloud ?
Reponse genere par le system :  Réponse: Un espace de stockage en ligne pour conserver des fichiers accessibles partout., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Qu'est-ce qu'un serveur ?
Reponse genere par le system :  Réponse: Un ordinateur qui stocke et partage des informations sur Internet., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  À quoi sert un mot de passe ?
Reponse genere par le system :  Réponse: À protéger l'accès à un compte personnel ou à des informations sensibles., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Comment fonctionne Internet ?
Reponse genere par le system :  Réponse: Internet fonctionne en connectant des ordinateurs entre eux grâce à un réseau mondial., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Qu'est-ce

## 5.2 Dev utilisateur

### 2.1 Question simple

In [ ]:
list_dev = [
    ("Qu'est-ce qu'une API ?", "Interface de programmation permettant aux applications de communiquer entre elles."),
    ("Quelle est la différence entre un tableau et une liste en Python ?", "Un tableau a une taille fixe, alors qu'une liste est dynamique."),
    ("Qu'est-ce qu'un framework ?", "Un ensemble d'outils et de bibliothèques permettant de développer des applications rapidement."),
    ("Quelle est la fonction de Git ?", "Git est un système de contrôle de version distribué."),
    ("Qu'est-ce qu'une base de données relationnelle ?", "Une base de données où les données sont organisées en tables."),
    ("À quoi sert un fichier `.env` ?", "Il est utilisé pour stocker des variables d'environnement sensibles."),
    ("Qu'est-ce que le versioning de code ?", "Le versioning permet de gérer les différentes versions d'un code source."),
    ("Quelle est la différence entre une méthode GET et POST en HTTP ?", "GET est utilisé pour récupérer des données, POST pour envoyer des données."),
    ("Que fait une instruction `if` en programmation ?", "Elle permet d'exécuter du code conditionnel."),
    ("Qu'est-ce que le principe DRY en développement ?", "DRY signifie 'Don't Repeat Yourself', éviter la duplication du code.")
]

In [ ]:
for question in list_dev:
  #print(question[0])
  reponse_reel = question[1]
  reponse_genere = ask_question("Dev", question[0])

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  Interface de programmation permettant aux applications de communiquer entre elles.
Reponse genere par le system :  Réponse: Interface de programmation permettant aux applications de communiquer entre elles., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Un tableau a une taille fixe, alors qu'une liste est dynamique.
Reponse genere par le system :  Réponse: Un tableau a une taille fixe, alors qu'une liste est dynamique., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Un ensemble d'outils et de bibliothèques permettant de développer des applications rapidement.
Reponse genere par le system :  Réponse: Un ensemble d'outils et de bibliothèques permettant de développer des applications rapidement., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Git est un système de contrôle de version distribué.
Reponse genere par le system :

### 2.2 Question simple avec une reformulation

In [ ]:
list_dev_re = [
    ("Qu'est-ce qu'une API et à quoi sert-elle ?", "Une API permet aux applications de communiquer entre elles."),
    ("Quelle distinction existe entre un tableau et une liste en Python ?", "Un tableau a une taille fixe, tandis qu'une liste peut être redimensionnée."),
    ("En quoi consiste un framework en développement ?", "Un framework est un ensemble d'outils pour faciliter le développement d'applications."),
    ("Comment Git facilite-t-il la gestion de versions ?", "Git permet de suivre les changements et de gérer les versions du code source."),
    ("Qu'est-ce qu'une base de données relationnelle et comment fonctionne-t-elle ?", "Une base de données relationnelle stocke des données sous forme de tables reliées entre elles."),
    ("À quoi sert un fichier .env dans un projet ?", "Un fichier .env contient des variables d'environnement, souvent utilisées pour stocker des secrets."),
    ("Qu'entend-on par versioning dans le développement de logiciels ?", "Le versioning permet de gérer différentes versions d'un projet de manière organisée."),
    ("Quelle est la différence entre les méthodes HTTP GET et POST ?", "GET est utilisé pour récupérer des données, tandis que POST est utilisé pour envoyer des données."),
    ("Que permet d'accomplir une instruction 'if' dans un programme ?", "L'instruction 'if' permet d'exécuter un bloc de code conditionnellement."),
    ("Quel principe est exprimé par l'acronyme DRY en développement logiciel ?", "DRY signifie 'Don't Repeat Yourself', il vise à éviter la duplication du code.")
]

In [ ]:
for question in list_dev_re:
  #print(question[0])
  reponse_reel = question[1]
  reponse_genere = ask_question("Dev", question[0])

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  Une API permet aux applications de communiquer entre elles.
Reponse genere par le system :  Réponse: Interface de programmation permettant aux applications de communiquer entre elles., Pertinence (distance): 0.3431
--------------------------------------------------
Reel reponse :  Un tableau a une taille fixe, tandis qu'une liste peut être redimensionnée.
Reponse genere par le system :  Réponse: Un tableau a une taille fixe, alors qu'une liste est dynamique., Pertinence (distance): 0.0761
--------------------------------------------------
Reel reponse :  Un framework est un ensemble d'outils pour faciliter le développement d'applications.
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  Git permet de suivre les changements et de gérer les versions du code source.
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
----------------

### 2.3 Question sur d'autre role

In [ ]:
for question in list_simple:
  #print(question[0])
  reponse_reel = question[1]
  reponse_genere = ask_question("Dev", question[0])

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  Paris
Reponse genere par le system :  Cette question associée à un autre rôle.
--------------------------------------------------
Reel reponse :  Océan Pacifique
Reponse genere par le system :  Cette question associée à un autre rôle.
--------------------------------------------------
Reel reponse :  7
Reponse genere par le system :  Cette question associée à un autre rôle.
--------------------------------------------------
Reel reponse :  Victor Hugo
Reponse genere par le system :  Cette question associée à un autre rôle.
--------------------------------------------------
Reel reponse :  Dollar américain
Reponse genere par le system :  Cette question associée à un autre rôle.
--------------------------------------------------
Reel reponse :  Kangourou
Reponse genere par le system :  Cette question associée à un autre rôle.
--------------------------------------------------
Reel reponse :  Portugais
Reponse genere par le system :  Cette question associée à un autre rôle

### 1.4 Question Hors context

In [ ]:
for question in list_questions_hors:
  #print(question[0])
  reponse_reel = question[1]
  reponse_genere = ask_question("Dev", question[0])

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  24 heures 39 minutes
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  Thomas Edison
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  La théorie qui décrit l'origine de l'univers à partir d'un point très dense et chaud.
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  La forêt amazonienne
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  Un phénomène où la Lune passe entre la Terre et le Soleil.
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  Le guépa

### 2.4 Questions communes

In [ ]:
list_questions_communes = [
    "Qu'est-ce que le cloud ?",
    "Qu'est-ce qu'un serveur ?",
    "À quoi sert un mot de passe ?",
    "Comment fonctionne Internet ?",
    "Qu'est-ce qu'un fichier ?"
]

In [ ]:
for question in list_questions_communes:
  #print(question[0])
  reponse_reel = question
  reponse_genere = ask_question("Dev", question)

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  Qu'est-ce que le cloud ?
Reponse genere par le system :  Réponse: Une infrastructure qui permet d'héberger et d'exécuter des applications à distance., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Qu'est-ce qu'un serveur ?
Reponse genere par le system :  Réponse: Une machine qui exécute des applications backend et gère les requêtes des clients., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  À quoi sert un mot de passe ?
Reponse genere par le system :  Réponse: À sécuriser l'accès aux bases de données, aux API et aux outils de développement., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Comment fonctionne Internet ?
Reponse genere par le system :  Réponse: Internet repose sur des protocoles comme HTTP et TCP/IP pour échanger des données entre serveurs et clients., Pertinence (distance): 0.0000
--------------------------

## 5.3 Admin utilisateur

### 3.1 Question simple

In [ ]:
list_admin = [
    ("Comment installer Apache sur un serveur Linux ?", "Utiliser la commande 'sudo apt install apache2' sur une machine Ubuntu."),
    ("Que faire en cas de 'Disk Full' sur un serveur Linux ?", "Libérer de l'espace disque en supprimant des fichiers inutiles."),
    ("Comment vérifier les journaux système sur un serveur Linux ?", "Utiliser la commande 'journalctl' pour consulter les journaux."),
    ("Comment configurer un pare-feu sur Ubuntu ?", "Utiliser 'ufw' pour configurer un pare-feu."),
    ("Quelle commande permet de redémarrer un service sous Linux ?", "Utiliser la commande 'sudo systemctl restart <service>'."),
    ("Comment ajouter un utilisateur sur un serveur Linux ?", "Utiliser la commande 'sudo adduser <nom_utilisateur>'."),
    ("Comment configurer un serveur DNS sur Linux ?", "Modifier le fichier '/etc/resolv.conf' pour spécifier les serveurs DNS."),
    ("Que fait la commande `chmod` ?", "Elle permet de modifier les permissions d'un fichier ou d'un répertoire."),
    ("Comment configurer un serveur Nginx pour une application web ?", "Modifier les fichiers de configuration dans '/etc/nginx/sites-available' et activer le site."),
    ("Que faire pour sécuriser un serveur Linux contre les attaques DDoS ?", "Utiliser des outils comme fail2ban et configurer un pare-feu adapté.")
]

In [ ]:
for question in list_admin:
  #print(question[0])
  reponse_reel = question[1]
  reponse_genere = ask_question("Admin", question[0])

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  Utiliser la commande 'sudo apt install apache2' sur une machine Ubuntu.
Reponse genere par le system :  Réponse: Utiliser la commande 'sudo apt install apache2' sur une machine Ubuntu., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Libérer de l'espace disque en supprimant des fichiers inutiles.
Reponse genere par le system :  Réponse: Libérer de l'espace disque en supprimant des fichiers inutiles., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Utiliser la commande 'journalctl' pour consulter les journaux.
Reponse genere par le system :  Réponse: Utiliser la commande 'journalctl' pour consulter les journaux., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Utiliser 'ufw' pour configurer un pare-feu.
Reponse genere par le system :  Réponse: Utiliser 'ufw' pour configurer un pare-feu., Pertinence (distance): 0.0000
---------

### 3.2 Question simple avec une reformulation

In [ ]:
list_admin_re = [
    ("Comment procéder pour installer Apache sur un serveur Linux ?", "Utilisez la commande 'sudo apt install apache2' sur une machine Ubuntu."),
    ("Que faire lorsqu'un serveur Linux affiche un message indiquant que l'espace disque est plein ?", "Libérez de l'espace disque en supprimant des fichiers inutiles ou en agrandissant la partition."),
    ("Comment accéder et analyser les journaux systèmes sur un serveur Linux ?", "Utilisez la commande 'journalctl' pour consulter les journaux système."),
    ("Quelle est la méthode pour configurer un pare-feu sur un serveur Ubuntu ?", "Utilisez 'ufw' pour activer et configurer le pare-feu sur Ubuntu."),
    ("Quelle commande faut-il utiliser pour redémarrer un service sur Linux ?", "La commande est 'sudo systemctl restart <service>'."),
    ("Quelle procédure faut-il suivre pour ajouter un utilisateur sur un serveur Linux ?", "Utilisez la commande 'sudo adduser <nom_utilisateur>' pour ajouter un utilisateur."),
    ("Comment configurer un serveur DNS sous Linux ?", "Modifiez le fichier '/etc/resolv.conf' pour spécifier les serveurs DNS."),
    ("Que permet de faire la commande chmod sur Linux ?", "La commande 'chmod' permet de modifier les permissions d'un fichier ou répertoire."),
    ("Comment mettre en place un serveur Nginx pour héberger une application web ?", "Modifiez les fichiers dans '/etc/nginx/sites-available' et activez le site avec un lien symbolique."),
    ("Quelles mesures prendre pour protéger un serveur Linux contre les attaques par déni de service (DDoS) ?", "Utilisez des outils comme fail2ban et configurez un pare-feu pour limiter les connexions malveillantes.")
]

In [ ]:
for question in list_admin_re:
  #print(question[0])
  reponse_reel = question[1]
  reponse_genere = ask_question("Admin", question[0])

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  Utilisez la commande 'sudo apt install apache2' sur une machine Ubuntu.
Reponse genere par le system :  Réponse: Utiliser la commande 'sudo apt install apache2' sur une machine Ubuntu., Pertinence (distance): 0.2242
--------------------------------------------------
Reel reponse :  Libérez de l'espace disque en supprimant des fichiers inutiles ou en agrandissant la partition.
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  Utilisez la commande 'journalctl' pour consulter les journaux système.
Reponse genere par le system :  Réponse: Utiliser la commande 'journalctl' pour consulter les journaux., Pertinence (distance): 0.1639
--------------------------------------------------
Reel reponse :  Utilisez 'ufw' pour activer et configurer le pare-feu sur Ubuntu.
Reponse genere par le system :  Réponse: Utiliser 'ufw' pour configurer un pare-feu., Pertinence (distance): 0.

### 3.3 Question sur d'autre role

In [ ]:
list_dev = [
    ("Qu'est-ce qu'une API ?", "Interface de programmation permettant aux applications de communiquer entre elles."),
    ("Quelle est la différence entre un tableau et une liste en Python ?", "Un tableau a une taille fixe, alors qu'une liste est dynamique."),
    ("Qu'est-ce qu'un framework ?", "Un ensemble d'outils et de bibliothèques permettant de développer des applications rapidement."),
    ("Quelle est la fonction de Git ?", "Git est un système de contrôle de version distribué."),
    ("Qu'est-ce qu'une base de données relationnelle ?", "Une base de données où les données sont organisées en tables."),
    ("À quoi sert un fichier `.env` ?", "Il est utilisé pour stocker des variables d'environnement sensibles."),
    ("Qu'est-ce que le versioning de code ?", "Le versioning permet de gérer les différentes versions d'un code source."),
    ("Quelle est la différence entre une méthode GET et POST en HTTP ?", "GET est utilisé pour récupérer des données, POST pour envoyer des données."),
    ("Que fait une instruction `if` en programmation ?", "Elle permet d'exécuter du code conditionnel."),
    ("Qu'est-ce que le principe DRY en développement ?", "DRY signifie 'Don't Repeat Yourself', éviter la duplication du code.")
]

In [ ]:
for question in list_dev:
  #print(question[0])
  reponse_reel = question[1]
  reponse_genere = ask_question("Admin", question[0])

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  Interface de programmation permettant aux applications de communiquer entre elles.
Reponse genere par le system :  Cette question associée à un autre rôle.
--------------------------------------------------
Reel reponse :  Un tableau a une taille fixe, alors qu'une liste est dynamique.
Reponse genere par le system :  Cette question associée à un autre rôle.
--------------------------------------------------
Reel reponse :  Un ensemble d'outils et de bibliothèques permettant de développer des applications rapidement.
Reponse genere par le system :  Cette question associée à un autre rôle.
--------------------------------------------------
Reel reponse :  Git est un système de contrôle de version distribué.
Reponse genere par le system :  Cette question associée à un autre rôle.
--------------------------------------------------
Reel reponse :  Une base de données où les données sont organisées en tables.
Reponse genere par le system :  Cette question associée à un autre 

### 1.4 Question Hors context

In [ ]:
for question in list_questions_hors:
  #print(question[0])
  reponse_reel = question[1]
  reponse_genere = ask_question("Admin", question[0])

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  24 heures 39 minutes
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  Thomas Edison
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  La théorie qui décrit l'origine de l'univers à partir d'un point très dense et chaud.
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  La forêt amazonienne
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  Un phénomène où la Lune passe entre la Terre et le Soleil.
Reponse genere par le system :  Cette question ne correspond à aucun des rôles définis.
--------------------------------------------------
Reel reponse :  Le guépa

### 2.4 Questions communes

In [ ]:
for question in list_questions_communes:
  #print(question[0])
  reponse_reel = question
  reponse_genere = ask_question("Admin", question)

  print('Reel reponse : ', reponse_reel)
  print('Reponse genere par le system : ', reponse_genere)
  print(50*'-')

Reel reponse :  Qu'est-ce que le cloud ?
Reponse genere par le system :  Réponse: Un ensemble de serveurs distants offrant des services de stockage, de calcul et de réseau., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Qu'est-ce qu'un serveur ?
Reponse genere par le system :  Réponse: Un système informatique qui fournit des services à d'autres machines dans un réseau., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  À quoi sert un mot de passe ?
Reponse genere par le system :  Réponse: À contrôler l'authentification des utilisateurs sur les systèmes et les serveurs., Pertinence (distance): 0.0000
--------------------------------------------------
Reel reponse :  Comment fonctionne Internet ?
Reponse genere par le system :  Réponse: Internet est basé sur une infrastructure réseau reliant des millions de serveurs et d'appareils via des routeurs et des commutateurs., Pertinence (distance):

# 6. Interrogez LLaMa-2-7b-chat pour obtenir des réponses contextuelles

In [29]:
#!pip install bitsandbytes transformers accelerate
#!pip install -U bitsandbytes

In [1]:
from huggingface_hub import login
login()

il est préférable de le quantifier pour optimiser les performances et réduire la consommation mémoire, surtout si tu tournes sur un GPU avec peu de VRAM.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Configuration pour charger le modèle en 4 bits
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # Utiliser float16 pour de meilleures perfs
    bnb_4bit_use_double_quant=True,  # Double quantization pour réduire encore plus la mémoire
    bnb_4bit_quant_type="nf4"  # nf4 est plus efficace que fp4
)

# Charger LLaMA-2-7B-Chat avec quantization
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
# Vérifier si le modèle est bien chargé sur GPU
print(model.hf_device_map)

In [ ]:
'''import torch

# Charger le modèle sur le GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Définir le prompt
prompt_test = "Réponds uniquement en français. Bonjour, comment allez-vous ?"

# Tokeniser l'entrée et envoyer sur le GPU
inputs = tokenizer(prompt_test, return_tensors="pt").to(device)

# Générer la réponse
outputs = model.generate(**inputs, max_length=100)

# Décoder et afficher la réponse
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)'''

In [3]:
import torch

# Charger le modèle sur le GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

def query_llama(question, reponse):
    # Format the input prompt
    #formatted_prompt = f"Question: {question}\n\nRespons: {response}"
    formatted_prompt = f"""
                              Voici une question posée par un utilisateur et la réponse récupérée à partir de ChromaDB.

                              🔹 **Instructions claires :**
                              - **Si la réponse récupérée est valide**, reformule-la en français avec plus de contexte et d'explications. Ajoute des émojis dynamiquement pour illustrer les concepts (ex : 🌍 pour un lieu, 🧠 pour une explication, ✅ pour une réponse correcte). Termine la réponse par ✅.
                              - **Si la réponse est exactement "Cette question ne correspond à aucun des rôles définis.", alors affiche exactement :**
                                **"Cette question ne correspond à aucun des rôles définis. ❌"**
                              - **Si la réponse est exactement " Cette question associée à un autre rôle. ", alors affiche exactement :**
                                **"Cette question est associée à un autre rôle. 🔄"**
                              - **Ne génère aucun autre texte en dehors des trois cas définis ci-dessus.**

                              **⚠️ Attention :** Tu dois suivre ces règles strictement et ne pas ajouter d'explications supplémentaires si la réponse ne correspond pas aux données disponibles.

                              **Longueur maximale de la réponse générée : 500 caractères.**

                              Question : {question}
                              Réponse récupérée : {reponse}
                              💡 Réponse en français :
                        """



    # Tokeniser l'entrée et envoyer sur le GPU
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

    # Générer la réponse
    outputs = model.generate(**inputs, max_length=800)

    # Décoder et afficher la réponse
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

## Question 1

In [ ]:
test_question = "Quelle est la capitale de la France ?"
response = ask_question("Simple", test_question)
print('response = ', response)

test_answer = query_llama(test_question, response)  # Utilisation de Mistral au lieu de DeepSeek

#print("✅ LLaMa-2-7b-chat a généré une réponse :", test_answer)
test_answer

In [ ]:
# Extraction de la réponse en français après "💡 Réponse en français :"
start_index = test_answer.find("💡 Réponse en français :") + len("💡 Réponse en français :")
filtered_response = test_answer[start_index:].strip()

# Enlever "Correcte !" à la fin si nécessaire
filtered_response = filtered_response.replace("Correcte", "").strip()

print("Réponse extraite :", filtered_response)

## Question 2

In [ ]:
test_question = "Qu'est-ce qu'une API ?"
response = ask_question("Dev", test_question)
print('response = ', response)

test_answer = query_llama(test_question, response)  # Utilisation de Mistral au lieu de DeepSeek

#print("✅ LLaMa-2-7b-chat a généré une réponse :", test_answer)
test_answer

In [ ]:
# Extraction de la réponse en français après "💡 Réponse en français :"
start_index = test_answer.find("💡 Réponse en français :") + len("💡 Réponse en français :")
filtered_response = test_answer[start_index:].strip()

# Enlever "Correcte !" à la fin si nécessaire
filtered_response = filtered_response.replace("Correct", "").strip()

print("Réponse extraite :", filtered_response)

## Question 3

In [ ]:
test_question = "Comment installer Apache sur un serveur Linux ?"
response = ask_question("Admin", test_question)
print('response = ', response)

test_answer = query_llama(test_question, response)  # Utilisation de Mistral au lieu de DeepSeek

#print("✅ LLaMa-2-7b-chat a généré une réponse :", test_answer)
test_answer

In [ ]:
# Extraction de la réponse en français après "💡 Réponse en français :"
start_index = test_answer.find("💡 Réponse en français (max 500 caractères) :") + len("💡 Réponse en français (max 500 caractères) :")
filtered_response = test_answer[start_index:].strip()

# Enlever "Correcte !" à la fin si nécessaire
filtered_response = filtered_response.replace("Correct", "").strip()

print("Réponse extraite :", filtered_response)

## Question 4 dans autre role

In [ ]:
test_question = "Quelle est la fonction de Git ?"
response = ask_question("Dev", test_question)

if response == message_question_existante_autre_role:
  print(response, '🔄')
elif response==message_hors_contexte:
  print(response, '❌')
else:
  print(response)
  test_answer = query_llama(test_question, response)

  # Extraction de la réponse en français après "💡 Réponse en français :"
  start_index = test_answer.find("💡 Réponse en français :") + len("💡 Réponse en français :")
  filtered_response = test_answer[start_index:].strip()

  # Enlever "Correcte !" à la fin si nécessaire
  filtered_response = filtered_response.replace("Correct", "").strip()

  print("Réponse extraite :", filtered_response)

# 7. Poser des question

In [23]:
def query(query:str, role:str):
  #print('query : ', query)
  #print('role : ', role)
  response = ask_question(role, query)
  print('response ask_question : ', response)

  if response == message_question_existante_autre_role:
    return response + ' 🔄'
  elif response==message_hors_contexte:
    return response + ' ❌'
  else:
    test_answer = query_llama(query, response)

    # Extraction de la réponse en français après "💡 Réponse en français :"
    start_index = test_answer.find("💡 Réponse en français :") + len("💡 Réponse en français :")
    filtered_response = test_answer[start_index:].strip()

    # Enlever "Correcte !" à la fin si nécessaire
    filtered_response = filtered_response.replace("Correct", "").strip()

    return filtered_response

## 7.1 Simple Utilisateur

In [ ]:
query("Quelle est la capitale de la France ?", "Simple")

response ask_question :  Paris


'La capitale de la France est Paris. Elle est située dans le nord-est du pays, au centre de la région Île-de-France. Paris est une ville historique et culturelle, connue pour ses monuments célèbres tels que la Tour Eiffel, la Cathédrale de Notre-Dame et le Louvre. La ville est également célèbre pour ses quartiers résidentiels, ses rues étroites et pittoresques, ainsi que pour ses nombreux parcs et jardins. ✅'

In [ ]:
query("Quel est le plus grand océan du monde ?", "Simple")

response ask_question :  Océan Pacifique


"Le plus grand océan du monde est l'Océan Pacifique, qui couvre une superficie de plus de 155 millions de kilomètres carrés. Il s'étend sur plus de 150° de longitude et sur une largeur de plus de 40° de latitude. L'Océan Pacifique est également le plus profond océan du monde, avec une profondeur maximale de plus de 11 000 mètres. ✅"

In [ ]:
query("Quelle est la langue officielle du Brésil ?", "Simple")

response ask_question :  Portugais


"Le Brésil a trois langues officielles : le portugais, le brésilien (ou portugeiro) et l'espagnol. Le portugais est la langue la plus largement parlée et utilisée dans le pays. 🇧🇷"

In [ ]:
query("Quelle est la hauteur du Mont Everest ?", "Simple")

response ask_question :  8 848 mètres


"Le Mont Everest, également connu sous le nom de Chomolungma ou Sagarmatha, est le sommet le plus élevé de la planète avec une hauteur de 8 848 mètres. Il se situe dans les Himalayas, entre le Népal et la Chine. Cette montagne est considérée comme l'un des plus beaux et des plus difficiles à escalader des sommets du monde, avec des pentes raides, des températures extrêmes et un environnement extrêmement hostile. ✅"

## 7.2 Dev Utilisateur

In [ ]:
query("Qu'est-ce qu'une API ?", "Dev")

response ask_question :  Interface de programmation permettant aux applications de communiquer entre elles.


"Une API (Interfaced'Programmation) est un ensemble de règles et de conventions qui permettent aux applications de communiquer entre elles. C'est un moyen pour un développeur de faire communiquer ses applications avec celles d'autres développeurs ou avec des services en ligne. Par exemple, une API pour une plateforme de livraison pourrait permettre à une application de demander un livre à être livré à un adresse spécifique. ✅"

In [ ]:
query("Quelle est la différence entre un tableau et une liste en Python ?", "Dev")

response ask_question :  Un tableau a une taille fixe, alors qu'une liste est dynamique.


"Un tableau est un ensemble d'éléments organisé dans une matrice fixe, tandis qu'une liste est un ensemble d'éléments qui peut croître ou diminuer en taille dynamiquement. 📈 En d'autres termes, un tableau est plus adapté pour stocker des données qui ne changent pas, tandis qu'une liste est plus flexible pour gérer des données qui peuvent varier. 🔄"

In [ ]:
query("Qu'est-ce qu'un framework ?", "Dev")

response ask_question :  Un ensemble d'outils et de bibliothèques permettant de développer des applications rapidement.


"Un framework est un ensemble d'outils et de bibliothèques qui facilite le développement d'applications rapido. Il permet de réutiliser des parties de code et de répondre à des problèmes de base de manière plus efficace. En d'autres termes, un framework est un outil qui permet de construire des applications plus rapidement et de manière plus efficace. 💻"

In [ ]:
query("Que fait une instruction `if` en programmation ?", "Dev")

response ask_question :  Elle permet d'exécuter du code conditionnel.


"L'instruction `if` en programmation permet de exécuter du code conditionnel. Cela signifie qu'elle permet de choisir si on veut exécuter un code ou un autre en fonction d'une condition. Par exemple, on peut utiliser l'instruction `if` pour vérifier si une variable contient un certain valeur, et si c'est le cas, exécuter un code différent de celui qui est exécuté si la variable n'a pas cette valeur. Cela permet de gérer les cas d'utilisation spécifiques et de personnaliser le code en fonction des besoins. ✅"

## 7.3 Admin Utilisateur

In [ ]:
query("Comment installer Apache sur un serveur Linux ?", "Admin")

response ask_question :  Utiliser la commande 'sudo apt install apache2' sur une machine Ubuntu.


'Installing Apache on a Linux server can be done using the `sudo apt install apache2` command on an Ubuntu machine. 🏗️ This command will install the Apache web server and its dependencies on your server. Once the installation is complete, you can configure the server using the Apache configuration files. 📝 To access the server, you can use a web browser or a command-line interface. ✅'

In [ ]:
query("Que faire en cas de 'Disk Full' sur un serveur Linux ?", "Admin")

response ask_question :  Libérer de l'espace disque en supprimant des fichiers inutiles.


'Si le serveur Linux est en train de montrer l\'erreur "Disk Full", cela signifie que le disque dur est complet et ne peut plus stocker de données supplémentaires. Pour résoudre ce problème, il est important de libérer de l\'espace disque en supprimant des fichiers inutiles ou en utilisant des outils de gestion de l\'espace disque pour déplacer des fichiers ou des directories vers un disque supplémentaire. Il est également important de surveiller les taille du disque et de prendre des mesures pour éviter que le disque ne soit plein à l\'avenir. ✅'

In [ ]:
query("Que fait la commande `chmod` ?", "Admin")

response ask_question :  Elle permet de modifier les permissions d'un fichier ou d'un répertoire.


"La commande `chmod` permet de modifier les permissions d'un fichier ou d'un répertoire sur un système Unix-like. Elle permet de définir ou de modifier les droits d'accès à un fichier ou un répertoire, en utilisant des codes numériques pour indiquer les types de droits d'accès souhaités (par exemple, lecturer, écrire, modifier, exécuter). La commande `chmod` est utile pour gérer les droits d'accès à des fichiers ou des répertoires à l'aide d'un interface simple et efficace. ✅"

## 7.4 D'autre role

In [ ]:
query("Quelle est la capitale de la France ?", "Dev")

response ask_question :  Cette question associée à un autre rôle.


'Cette question associée à un autre rôle. 🔄'

In [ ]:
query("Qu'est-ce qu'une API ?", "Admin")

response ask_question :  Cette question associée à un autre rôle.


'Cette question associée à un autre rôle. 🔄'

In [ ]:
query("Comment installer Apache sur un serveur Linux ?", "Simple")

response ask_question :  Cette question associée à un autre rôle.


'Cette question associée à un autre rôle. 🔄'

## 7.5 Hors context

In [ ]:
query("Quelle est la durée d'un jour sur Mars ?", "Simple")

response ask_question :  Cette question ne correspond à aucun des rôles définis.


'Cette question ne correspond à aucun des rôles définis. ❌'

In [ ]:
query("Quelle est la durée d'un jour sur Mars ?", "Dev")

response ask_question :  Cette question ne correspond à aucun des rôles définis.


'Cette question ne correspond à aucun des rôles définis. ❌'

In [ ]:
query("Quelle est la durée d'un jour sur Mars ?", "Admin")

response ask_question :  Cette question ne correspond à aucun des rôles définis.


'Cette question ne correspond à aucun des rôles définis. ❌'

## 7.6 Question communes

In [ ]:
query("Qu'est-ce que le cloud ?", "Simple")

response ask_question :  Un espace de stockage en ligne pour conserver des fichiers accessibles partout.


"Le cloud est un espace de stockage en ligne qui permet de stocker des fichiers à distance. Cela signifie qu'ils sont accessibles depuis n'importe quel endroit, à condition d'avoir une connexion internet. Les fichiers stockés dans le cloud peuvent être utilisés pour des tâches telles que la sauvegarde de données, la collaboration avec des partenaires ou la mise en place de services en nuage. ✅"

In [ ]:
query("Qu'est-ce que le cloud ?", "Dev")

response ask_question :  Une infrastructure qui permet d'héberger et d'exécuter des applications à distance.


"Le cloud est une infrastructure qui permet d'héberger et d'exécuter des applications à distance. C'est une solution qui permet de déplacer les logiciels et les données vers un serveur distant, ce qui permet d'accéder à ces derniers depuis n'importe où et à n'importe quelle époque. Cela permet également de gérer les ressources informatiques de manière plus efficace et de réduire les coûts d'équipement. ✅"

In [ ]:
query("Qu'est-ce que le cloud ?", "Admin")

response ask_question :  Un ensemble de serveurs distants offrant des services de stockage, de calcul et de réseau.


'Le "cloud" (ou "nuage") désigne un ensemble de serveurs distants qui offrent des services de stockage, de calcul et de réseau. Cela signifie qu\'au lieu de stocker des données sur un disque dure local ou sur un serveur physique, on peut les stocker dans un "nuage" de serveurs distants, ce qui permet de gagner en sécurité, en rapidité et en flexibilité. Par exemple, si vous utilisez une application en ligne, vous pouvez facilement accéder à elle depuis n\'importe quelle machine avec une connexion internet, sans avoir à installer de logiciels supplémentaires. ✅'

# 8. Gradio Interface

In [2]:
#!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 56.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [24]:
import gradio as gr

# Création de l'interface Gradio
interface = gr.Interface(
    fn=query,  # Appelle la fonction query(question, role)
    inputs=[
        gr.Textbox(label="Question"),  # Champ pour la question
        gr.Dropdown(["Simple", "Dev", "Admin"], label="Rôle")  # Sélection du rôle
    ],
    outputs="text",
    title="RAG Chatbot : Réponses adaptées selon le rôle",
    description="Pose une question et sélectionne un rôle (Simple, Dev ou Admin) pour obtenir une réponse personnalisée générée par LLaMa avec ChromaDB."
)

In [25]:
interface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://48e836ec8acb8ed30d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


response ask_question :  Paris
response ask_question :  Paris
response ask_question :  Cette question associée à un autre rôle.
response ask_question :  Interface de programmation permettant aux applications de communiquer entre elles.
response ask_question :  Une infrastructure qui permet d'héberger et d'exécuter des applications à distance.
response ask_question :  Un espace de stockage en ligne pour conserver des fichiers accessibles partout.
response ask_question :  Un ensemble de serveurs distants offrant des services de stockage, de calcul et de réseau.
response ask_question :  Cette question ne correspond à aucun des rôles définis.
response ask_question :  Cette question associée à un autre rôle.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://48e836ec8acb8ed30d.gradio.live
